In [2]:
import operator
import spacy
nlp = spacy.load("en_core_web_md")


In [3]:
docmap={}
topklist = [3,4,5]

In [ ]:
# irlist = ["data/wordintersection/score-val-top10-merged-val.tsv","data/wordintersection/score-val-val-trained.tsv"]
# irlist.extend(["data/wordunion/score-val-ir-top10-merged-val-wordunion.tsv","data/wordunion/score-val-val-trained-wordunion.tsv"])
# irlist.extend(["data/bagofwords/score-val-ir-top10-merged-bow-val.tsv","data/bagofwords/score-val-ir-not-merged-bow-val.tsv"])
# irlist.extend(["data/seq2seq/score-val-ir-top10-merged-val-seq2seq.tsv","data/seq2seq/score-val-ir-val-trained-seq2seq.tsv"])

# scorelist = ["data/wordintersection/score-val-top10-merged-val.tsv-score.tsv","data/wordintersection/score-val-val-trained.tsv-score.tsv"]
# scorelist.extend(["data/wordunion/score-val-ir-top10-merged-val-wordunion.tsv-score.tsv","data/wordunion/score-val-val-trained-wordunion.tsv-score.tsv"])
# scorelist.extend(["data/bagofwords/score-val-ir-top10-merged-bow-val.tsv-score.tsv","data/bagofwords/score-val-ir-not-merged-bow-val.tsv-score.tsv"])
# scorelist.extend(["data/seq2seq/score-val-ir-top10-merged-val-seq2seq.tsv-score.tsv","data/seq2seq/score-val-ir-val-trained-seq2seq.tsv-score.tsv"])

# picklist = ["data/wordintersection/ir-top10-val-merged-intersect.pickle","data/wordintersection/ir-val-trained-intersect.pickle"]
# picklist.extend(["data/wordunion/ir-top10-merged-val-union-merged.pickle","data/wordunion/val-trained-merged.pickle"])
# picklist.extend(["data/bagofwords/ir-top10-merged-bow.pickle","data/bagofwords/ir-not-merged-bow.pickle"])
# picklist.extend(["data/seq2seq/ir-top10-merged-val-seq2seq.pickle","data/seq2seq/ir-val-trained-seq2seq.pickle"])

In [87]:
irlist = ["data/wordintersection/score-test-top10-merged-test.tsv","data/wordintersection/score-test-test-trained.tsv"]
irlist.extend(["data/wordunion/score-test-ir-top10-merged-test-wordunion.tsv","data/wordunion/score-test-test-trained-wordunion.tsv"])
irlist.extend(["data/bagofwords/score-test-ir-top10-merged-bow-test.tsv","data/bagofwords/score-test-ir-not-merged-bow-test.tsv"])
irlist.extend(["data/seq2seq/score-test-ir-top10-merged-test-seq2seq.tsv","data/seq2seq/score-test-ir-test-trained-seq2seq.tsv"])

scorelist = ["data/wordintersection/score-test-top10-merged-test.tsv-score.tsv","data/wordintersection/score-test-test-trained.tsv-score.tsv"]
scorelist.extend(["data/wordunion/score-test-ir-top10-merged-test-wordunion.tsv-score.tsv","data/wordunion/score-test-test-trained-wordunion.tsv-score.tsv"])
scorelist.extend(["data/bagofwords/score-test-ir-top10-merged-bow-test.tsv-score.tsv","data/bagofwords/score-test-ir-not-merged-bow-test.tsv-score.tsv"])
scorelist.extend(["data/seq2seq/score-test-ir-top10-merged-test-seq2seq.tsv-score.tsv","data/seq2seq/score-test-ir-test-trained-seq2seq.tsv-score.tsv"])

picklist = ["data/wordintersection/ir-top10-test-merged-intersect.pickle","data/wordintersection/ir-test-trained-intersect.pickle"]
picklist.extend(["data/wordunion/ir-top10-merged-test-union-merged.pickle","data/wordunion/test-trained-merged.pickle"])
picklist.extend(["data/bagofwords/ir-top10-merged-bow.pickle","data/bagofwords/ir-not-merged-bow.pickle"])
picklist.extend(["data/seq2seq/ir-top10-merged-test-seq2seq.pickle","data/seq2seq/ir-test-trained-seq2seq.pickle"])

In [85]:
testlines = open("../data/orig/orig-ques-test.tsv").readlines()
labelsdict = {}
for line in testlines:
    line=line.strip().split("\t")
    qid = line[0]
    label = line[-1]
    labelsdict[qid]=int(label)
    choices = line[2:6]
    for index,choice in enumerate(choices):
        labelsdict[qid+":"+str(index)]=choice

In [6]:
import nltk
from nltk.corpus import stopwords
stpwords =  set(stopwords.words('english'))

In [7]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

In [90]:
from tqdm import tqdm,tqdm_notebook
import string

def strip_punct(s):
    exclude = set(string.punctuation)
    s = ''.join(ch for ch in s if ch not in exclude)
    return s


def get_doc(docs):
    if docs in docmap:
        return docmap[docs]
    docmap[docs] = nlp(docs)
    return docmap[docs]

def process_string(inp):
    inp = strip_punct(inp)
    inp = inp.lower()
    uniq_words = set(inp.split(" "))
    return set(map(stemmer.stem,uniq_words - stpwords))

def is_same_as_ans(choice,f2):
    f2 = set(map(stemmer.stem,strip_punct(f2).lower().split(" "))) -set([''])
    choice = set(map(stemmer.stem,strip_punct(choice).lower().split(" "))) -set([''])
    return len(f2-choice) == 0 and len(choice-f2) == 0
    
def allow_add(eligible_s,all_entries):
    verbose=False
    ret = True
    eligible = process_string(eligible_s) - set([''])
    
    if len(eligible) <2:
        if verbose:
            print("Less Than 2!!",eligible,eligible_s)
        ret =  False
    for tup in all_entries.items():
        already_present = process_string(tup[0]) - set([''])
        if len(eligible - already_present) < 2:
            not_present = eligible - already_present
            if len(not_present) == 0:
                return False
            sim = 0
            for word1 in already_present:
                temp = 0
                for word2 in not_present:
                    temp = max(sim,get_doc(word1).similarity(get_doc(word2)))
                sim += temp
            
            if sim/len(not_present) > 0.5 :
                if verbose: print("Failed Allowed Add:",eligible,already_present,tup[0],eligible_s)
                ret = False
                break
    
    if verbose:
        print(eligible,eligible_s,ret)
            
    return ret
    
    
def rerank(tups,topkfacts):    
    result = {}
    src = dict(tups)
    if len(tups)==0:
        print(tups)
        return tups
    
    for tup in topkfacts:
        src[tup[0]]=float(tup[1])/20.0
    
    result[strip_punct(topkfacts[0][0])]=float(topkfacts[0][1])/20.0
    cur = get_doc(strip_punct(topkfacts[0][0]))
    src.pop(topkfacts[0][0])
    information_gains = {}
    while True:
        temp={}
        for tup in src.items():
            s = strip_punct(tup[0])
            doc = get_doc(s)
            information = cur.similarity(doc)
            information_gains[s] = max(information,information_gains.get(s,0))
        for tup in src.items():
            s = strip_punct(tup[0])
            temp[tup[0]]=tup[1]*(1-information_gains.get(s,0))
        if len(temp) == 0:
            break
        sorted_x = sorted(temp.items(), key=operator.itemgetter(1))
        
        for tup in reversed(sorted_x):
            eligible_string = strip_punct(tup[0])
            val = src.pop(tup[0]) 
            
            if allow_add(eligible_string,result):
                result[eligible_string]=val
                cur = get_doc(eligible_string)
                break
        
        if len(result)>15:
            break    
    sorted_x = sorted(result.items(), key=operator.itemgetter(1))
    return [i for i in reversed(sorted_x)]
    

def create_dataset(irtestlines,scorelines,labelsdict,topk):
    ir_dataset_f2= {}
    for irline,scoreline in tqdm(zip(irtestlines,scorelines)):
        scoreline = scoreline.strip().split("\t")
        qidx = scoreline[0].split(":")
        score =float(scoreline[-1])
        qid = qidx[0]
        hid = qidx[1]
        qidx = qid+":"+hid
        if qidx not in ir_dataset_f2:
            ir_dataset_f2[qidx]=ir_dataset[qidx]
            ir_dataset_f2[qidx]["f2set"]={}
        irline = irline.strip().split("\t")
        f2 = irline[2]
        choice = labelsdict[qidx]
        chdoc =get_doc(choice)
        score = score*chdoc.similarity(get_doc(f2))
        if is_same_as_ans(choice,f2):
            score= 0
        ir_dataset_f2[qidx]["f2set"][f2]= score

    ix=0
    topkf1=topk
    for qidx,row in tqdm(ir_dataset_f2.items()):
        sorted_x = sorted(row["f2set"].items(), key=operator.itemgetter(1))
        row["f2set"] = rerank([i for i in reversed(sorted_x)],row["f1list"][0:topkf1])
        ix+=1
    return ir_dataset_f2

In [94]:
import pickle

for topk in topklist:
    for irfile,scorefile,pickfile in zip(irlist,scorelist,picklist):
        print("Current File:",scorefile+'.repickled.'+str(topk))
        irtestlines = open(irfile).readlines()
        scorelines = open(scorefile).readlines()
        fd = open(pickfile,"rb")
        ir_dataset = pickle.load(fd)
        ir_dataset_reranked = create_dataset(irtestlines,scorelines,labelsdict,topk)
        with open(scorefile+'.repickled.'+str(topk), 'wb+') as handle:
            pickle.dump(ir_dataset_reranked, handle, protocol=pickle.HIGHEST_PROTOCOL)



0it [00:00, ?it/s]


Current File: data/wordintersection/score-test-top10-merged-test.tsv-score.tsv.repickled.3


258it [00:00, 2568.70it/s]
492it [00:00, 2494.73it/s]
813it [00:00, 2671.95it/s]
996it [00:00, 2344.49it/s]
1342it [00:00, 2594.02it/s]
1615it [00:00, 2632.74it/s]
1859it [00:00, 2319.70it/s]
2095it [00:00, 2330.55it/s]
2434it [00:00, 2571.53it/s]
2777it [00:01, 2778.91it/s]
3063it [00:01, 2785.71it/s]
3384it [00:01, 2898.48it/s]
3679it [00:01, 2624.50it/s]
3950it [00:01, 1981.41it/s]
4183it [00:01, 2074.38it/s]
4496it [00:01, 2305.41it/s]
4860it [00:01, 2590.29it/s]
5201it [00:01, 2789.85it/s]
5504it [00:02, 2821.95it/s]
5803it [00:02, 2807.84it/s]
6112it [00:02, 2883.79it/s]
6410it [00:02, 2714.85it/s]
6718it [00:02, 2814.44it/s]
7007it [00:02, 2503.93it/s]
7269it [00:02, 2114.29it/s]
7499it [00:02, 2036.03it/s]
7719it [00:03, 2081.58it/s]
8036it [00:03, 2318.19it/s]
8370it [00:03, 2552.05it/s]
8647it [00:03, 2613.61it/s]
9011it [00:03, 2853.44it/s]
9312it [00:03, 2584.38it/s]
9587it [00:03, 2627.69it/s]
9862it [00:03, 2662.32it/s]
10141it [00:03, 2697.50it/s]
10449it [00:04, 2801.72

Current File: data/wordintersection/score-test-test-trained.tsv-score.tsv.repickled.3


248it [00:00, 2479.92it/s]
464it [00:00, 2329.58it/s]
733it [00:00, 2425.50it/s]
925it [00:00, 2234.69it/s]
1230it [00:00, 2427.99it/s]
1468it [00:00, 2412.77it/s]
1749it [00:00, 2518.79it/s]
1986it [00:00, 2234.45it/s]
2288it [00:00, 2422.70it/s]
2622it [00:01, 2639.97it/s]
2935it [00:01, 2770.02it/s]
3242it [00:01, 2840.89it/s]
3531it [00:01, 2799.82it/s]
3815it [00:01, 2579.20it/s]
4079it [00:01, 2165.62it/s]
4347it [00:01, 2295.73it/s]
4695it [00:01, 2556.30it/s]
5073it [00:01, 2831.24it/s]
5384it [00:02, 2908.99it/s]
5691it [00:02, 2843.06it/s]
6021it [00:02, 2964.52it/s]
6327it [00:02, 2766.29it/s]
6621it [00:02, 2813.74it/s]
6910it [00:02, 2595.90it/s]
7178it [00:02, 2147.88it/s]
7412it [00:02, 2048.10it/s]
7631it [00:03, 1985.69it/s]
7843it [00:03, 2022.59it/s]
8183it [00:03, 2301.03it/s]
8466it [00:03, 2437.09it/s]
8847it [00:03, 2731.05it/s]
9143it [00:03, 2593.60it/s]
9420it [00:03, 2370.13it/s]
9730it [00:03, 2548.17it/s]
10000it [00:03, 2562.85it/s]
10294it [00:04, 2662.51

...


466it [00:00, 4653.53it/s]
890it [00:00, 4519.20it/s]
1268it [00:00, 4267.38it/s]
1583it [00:00, 3855.56it/s]
1961it [00:00, 3831.54it/s]
2329it [00:00, 3784.18it/s]
2812it [00:00, 4046.61it/s]
3362it [00:00, 4392.57it/s]
3790it [00:00, 4119.58it/s]
4197it [00:01, 3752.76it/s]
4608it [00:01, 3852.19it/s]
5095it [00:01, 4108.98it/s]
5555it [00:01, 4244.10it/s]
5986it [00:01, 4229.88it/s]
6413it [00:01, 3789.81it/s]
6817it [00:01, 3856.73it/s]
7211it [00:01, 3406.36it/s]
7610it [00:01, 3562.48it/s]
8035it [00:02, 3741.17it/s]
8421it [00:02, 3744.21it/s]
8825it [00:02, 3824.84it/s]
9214it [00:02, 3569.00it/s]
9579it [00:02, 3542.45it/s]
9941it [00:02, 3565.34it/s]
10340it [00:02, 3682.74it/s]
10719it [00:02, 3711.63it/s]
11096it [00:02, 3728.15it/s]
11471it [00:02, 3631.39it/s]
11847it [00:03, 3666.54it/s]
12216it [00:03, 3562.74it/s]
12687it [00:03, 3842.71it/s]
13132it [00:03, 4004.24it/s]
13540it [00:03, 3998.31it/s]
13945it [00:03, 3777.00it/s]
14329it [00:03, 3669.16it/s]
14742it [0

Current File: data/seq2seq/score-test-ir-top10-merged-test-seq2seq.tsv-score.tsv.repickled.5



521it [00:00, 5208.97it/s]
1063it [00:00, 5268.50it/s]
1598it [00:00, 5288.40it/s]
2102it [00:00, 5206.72it/s]
2585it [00:00, 5085.54it/s]
3100it [00:00, 5104.65it/s]
3550it [00:00, 4573.45it/s]
4079it [00:00, 4766.93it/s]
4670it [00:00, 5058.54it/s]
5167it [00:01, 4716.00it/s]
5679it [00:01, 4829.41it/s]
6162it [00:01, 4411.16it/s]
6692it [00:01, 4639.78it/s]
7198it [00:01, 4758.03it/s]
7680it [00:01, 4723.10it/s]
8203it [00:01, 4863.82it/s]
8843it [00:01, 5240.62it/s]
9379it [00:01, 4976.01it/s]
9888it [00:02, 4667.25it/s]
10367it [00:02, 4702.75it/s]
10958it [00:02, 5006.08it/s]
11470it [00:02, 4907.41it/s]
12011it [00:02, 5047.53it/s]
12523it [00:02, 4803.47it/s]
13011it [00:02, 4376.99it/s]
13522it [00:02, 4571.73it/s]
14040it [00:02, 4737.03it/s]
14559it [00:02, 4863.70it/s]
15070it [00:03, 4935.02it/s]
15569it [00:03, 4860.28it/s]
16060it [00:03, 4655.74it/s]
16531it [00:03, 4536.60it/s]
17127it [00:03, 4886.16it/s]
17627it [00:03, 4874.73it/s]
18122it [00:03, 4651.78it/s]
1859

Current File: data/seq2seq/score-test-ir-test-trained-seq2seq.tsv-score.tsv.repickled.5



529it [00:00, 5283.88it/s]
1053it [00:00, 5263.86it/s]
1591it [00:00, 5295.19it/s]
2089it [00:00, 5193.87it/s]
2551it [00:00, 5006.73it/s]
3094it [00:00, 5124.25it/s]
3547it [00:00, 4601.70it/s]
4035it [00:00, 4680.68it/s]
4630it [00:00, 4998.67it/s]
5121it [00:01, 4675.09it/s]
5630it [00:01, 4791.02it/s]
6109it [00:01, 4337.12it/s]
6656it [00:01, 4623.46it/s]
7152it [00:01, 4718.71it/s]
7632it [00:01, 4549.62it/s]
8122it [00:01, 4647.49it/s]
8766it [00:01, 5070.15it/s]
9289it [00:01, 4869.95it/s]
9789it [00:02, 4665.30it/s]
10266it [00:02, 4628.74it/s]
10846it [00:02, 4926.06it/s]
11352it [00:02, 4964.47it/s]
11856it [00:02, 4911.80it/s]
12353it [00:02, 4757.03it/s]
12834it [00:02, 4238.60it/s]
13321it [00:02, 4409.57it/s]
13869it [00:02, 4683.56it/s]
14351it [00:03, 4632.97it/s]
14892it [00:03, 4841.28it/s]
15385it [00:03, 4727.22it/s]
15865it [00:03, 4713.83it/s]
16342it [00:03, 4505.21it/s]
16897it [00:03, 4773.99it/s]
17383it [00:03, 4652.67it/s]
17855it [00:03, 4533.75it/s]
1831

In [84]:
ir_dataset_f2["7-1128:3"]["f2set"],ir_dataset_f2["605:2"]["f2set"],ir_dataset_f2["8-201:2"]["f2set"],ir_dataset_f2["9-1090:3"]["f2set"]

([('a seismograph is used for measuring the size of an earthquake',
   0.9535426499999999),
  ('Answer A seismograph is the device that scientists use to measure earthquakes',
   0.7816148108636974),
  ('Construct and use a seismograph to measure an quotearthquake',
   0.7761641946431864),
  ('Magnitude A measure of the strength of an earthquake or strain energy released by it as determined by seismographic observations',
   0.7625586133665916),
  ('Seismograph An instrument used to measure the shaking caused by an earthquake',
   0.7468490281113347),
  ('The Seismographs Earthquakes caused by platetectonic forces in the Earths interior  always produce seismic waves that can be recorded on sensitive instruments called seismographs ',
   0.744385826109664),
  ('Describe how seismographs measure earthquake strength and location',
   0.7171070000280524),
  ('The seismograph is introduced for the measurement of earthquakes',
   0.7164852837508432),
  ('Seismographs are instruments used to 

In [89]:
get_doc("Use the principles of camouflage.").similarity(get_doc("Using proper camouflage techniques."))

0.8411261022871526

In [38]:
for key in list(ir_dataset_f2.keys())[200:216]:
    print(ir_dataset_f2[key]["f2set"],ir_dataset_f2[key]["hyp"])

[('bears eat berries', 0.9462301999999999), ('a berry contains seeds', 0.9217318000000001), ('rabbits eat plants', 0.9057109999999999), ('only Brown Bears eat peoplealthough a Black Bear will trade you to a Brown Bear for some berries', 0.8451379337831414), ('Some varieties may take even a few years to flower and bear fruits', 0.7970812663325126), ('This gets back to the part about bears not eating berries', 0.7883606061189473), ('The seeds may get caught in the fur and transported to another area by the animal', 0.782675564413755), ('The rabbit a herbivore may eat the plant', 0.7586633072369164), ('Some flower and bear fruit', 0.707685035852894), ('They may eat parts of plants such as berries seeds or roots', 0.7073181779749377), ('The seeds are dispersed by sticking to animal fur', 0.6954554438409652), ('Some are flowering and some are fruitbearing', 0.6771528421773105), ('The local bears are finally eating berries instead of garbage', 0.6719016639154373), ('Let the birds eat the ber

In [31]:
u = "A best causes decide afford nice phone After make calls end less saving wants looking resources year. save budget way usually money using saved person vacation expenses, start"
i = "best using start end looking less usually saved vacation way nice resources wants phone person money causes year expenses make saving afford budget save decide calls"

In [33]:
u = set(u.split(" "))
i = set(i.split(" "))

In [42]:
f1 = "using less resources usually causes money to be saved"
h1 = "A person wants to start saving money so that they can afford a nice vacation at the end of the year. After looking over their budget and expenses, they decide the best way to save money is to make more phone calls"

In [43]:
f1 = set(f1.lower().split(" "))
h1 = set(h1.lower().split(" "))

In [46]:
(f1.union(h1) - f1.intersection(h1)) - stpwords

{'afford',
 'best',
 'budget',
 'calls',
 'causes',
 'decide',
 'end',
 'expenses,',
 'less',
 'looking',
 'make',
 'nice',
 'person',
 'phone',
 'resources',
 'save',
 'saved',
 'saving',
 'start',
 'using',
 'usually',
 'vacation',
 'wants',
 'way',
 'year.'}

In [47]:
f1.intersection(h1)

{'money', 'to'}